In [178]:
import numpy as np
import netCDF4 as nc
import auxiliary as au
import regression as reg
import datetime as dt
from matplotlib import pyplot as plt
from scipy import io
from scipy.interpolate import interp2d
import os
import sys
from scipy.interpolate import griddata
import h5py


def demread(file, lattar, lontar):
    datatemp = io.loadmat(file)
    demori = datatemp['DEM']
    demori[np.isnan(demori)] = 0
    info = datatemp['Info'][0][0]
    latori = np.arange(info['yll'] + info['Ysize'] * info['nrows'] - info['Ysize'] / 2, info['yll'], -info['Ysize'])
    lonori = np.arange(info['xll'] + info['Xsize'] / 2, info['xll'] + info['Xsize'] * info['ncols'], info['Xsize'])
    f = interp2d(lonori, latori, demori, kind='linear')
    demtar = f(lontar.flatten(), lattar.flatten())
    demtar = np.flipud(demtar)
    return demtar


def ncread(file, var):
    # read a variable from netcdf
    ncfid = nc.Dataset(file)
    data = ncfid[var][:].data
    ncfid.close()
    return data


def neargrid(rowtar, coltar, rowori, colori, hwsize):
    # inputs are 1D matrices
    # tar is target area
    # ori is original area
    # hwsize is half window size (e.g., 4 means the space window width/length is 2*4+1)
    # find a space window centering the target grid in the original area and calculate the weights
    nrows = len(rowtar)
    ncols = len(coltar)
    rowse = np.zeros([nrows, ncols, 2]).astype(int)  # se: start/end
    colse = np.zeros([nrows, ncols, 2]).astype(int)  # se: start/end
    weight = np.nan * np.zeros([nrows, ncols, (hwsize * 2 + 1) ** 2])  # from left to right/from top to bottom weight

    for rr in range(nrows):
        rowloc = np.argmin(np.abs(rowori - rowtar[rr]))
        rowse[rr, :, 0] = rowloc - hwsize
        rowse[rr, :, 1] = rowloc + hwsize

    for cc in range(ncols):
        colloc = np.argmin(np.abs(colori - coltar[cc]))
        colse[:, cc, 0] = colloc - hwsize
        colse[:, cc, 1] = colloc + hwsize

    rowse[rowse < 0] = 0
    rowse[rowse > nrows] = nrows
    colse[colse < 0] = 0
    colse[colse > ncols] = nrows

    maxdist = (hwsize + 0.5) * np.sqrt(2) + 0.5
    for rr in range(nrows):
        rowloc = np.argmin(np.abs(rowori - rowtar[rr]))
        for cc in range(ncols):
            colloc = np.argmin(np.abs(colori - coltar[cc]))

            rowse_rc = rowse[rr, cc, :]
            colse_rc = colse[rr, cc, :]
            flag = 0
            for i in range(rowse_rc[0], rowse_rc[1] + 1):
                for j in range(colse_rc[0], colse_rc[1] + 1):
                    dist = ((rowloc - i) ** 2 + (colloc - j) ** 2) ** 0.5
                    weight[rr, cc, flag] = au.distanceweight(dist, maxdist, 3)
                    flag = flag + 1

            weight[rr, cc, :] = weight[rr, cc, :] / np.nansum(weight[rr, cc, :])

    return rowse, colse, weight


def readownscale(dataori, latori, lonori, demori, lattar, lontar, demtar, rowse, colse, weight, mask):
    nrows = len(lattar)
    ncols = len(lontar)
    ntimes = np.shape(dataori)[2]
    lonori, latori = np.meshgrid(lonori, latori)
    datatar = np.nan * np.zeros([nrows, ncols, ntimes])

    for rr in range(nrows):
        for cc in range(ncols):
            if mask[rr, cc] == 1:
                rloc = rowse[rr, cc, :]
                cloc = colse[rr, cc, :]
                latnear = latori[rloc[0]:rloc[1] + 1, cloc[0]:cloc[1] + 1]
                lonnear = lonori[rloc[0]:rloc[1] + 1, cloc[0]:cloc[1] + 1]
                demnear = demori[rloc[0]:rloc[1] + 1, cloc[0]:cloc[1] + 1]
                nnum = np.size(latnear)
                latnear = np.reshape(latnear, nnum)
                lonnear = np.reshape(lonnear, nnum)
                demnear = np.reshape(demnear, nnum)
                weightnear = np.zeros([nnum, nnum])
                for i in range(nnum):
                    weightnear[i, i] = weight[rr, cc, i]

                nearinfo = np.zeros([nnum, 4])
                nearinfo[:, 0] = 1
                nearinfo[:, 1] = latnear
                nearinfo[:, 2] = lonnear
                nearinfo[:, 3] = demnear

                tarinfo = np.zeros(4)
                tarinfo[0] = 1
                tarinfo[1] = lattar[rr]
                tarinfo[2] = lontar[cc]
                tarinfo[3] = demtar[rr, cc]

                tx_red = np.transpose(nearinfo)
                twx_red = np.matmul(tx_red, weightnear)

                for tt in range(ntimes):
                    datanear = dataori[rloc[0]:rloc[1] + 1, cloc[0]:cloc[1] + 1, tt]
                    datanear = np.reshape(datanear, nnum)

                    # upper and lower boundary for the downscaled data
                    # this is a conservative limitation
                    lowbound = np.min(datanear)
                    upbound = np.max(datanear)

                    b = reg.least_squares(nearinfo, datanear, twx_red)
                    datatemp = np.dot(tarinfo, b)
                    if np.all(b == 0) or datatemp>upbound or datatemp<lowbound:
                        # use nearest neighbor interpolation
                        weightnear = weight[rr, cc, 0:nnum]
                        mloc = np.argmax(weightnear)
                        datatar[rr, cc, tt] = datanear[mloc]
                    else:
                        datatar[rr, cc, tt] = datatemp
    return datatar

def readownscale_tostn(dataori, latori, lonori, demori, lattar, lontar, demtar, rowse, colse, weight, stn_row, stn_col):
    nstn = len(stn_row)
    ntimes = np.shape(dataori)[2]
    lonori, latori = np.meshgrid(lonori, latori)
    datatar = np.nan * np.zeros([nstn, ntimes])

    for gg in range(nstn):
        if np.mod(gg,500)==0:
            print('station', gg, nstn)
        rr = stn_row[gg]
        cc = stn_col[gg]
        rloc = rowse[rr, cc, :]
        cloc = colse[rr, cc, :]
        latnear = latori[rloc[0]:rloc[1] + 1, cloc[0]:cloc[1] + 1]
        lonnear = lonori[rloc[0]:rloc[1] + 1, cloc[0]:cloc[1] + 1]
        demnear = demori[rloc[0]:rloc[1] + 1, cloc[0]:cloc[1] + 1]
        nnum = np.size(latnear)
        latnear = np.reshape(latnear, nnum)
        lonnear = np.reshape(lonnear, nnum)
        demnear = np.reshape(demnear, nnum)
        weightnear = np.zeros([nnum, nnum])
        for i in range(nnum):
            weightnear[i, i] = weight[rr, cc, i]

        nearinfo = np.zeros([nnum, 4])
        nearinfo[:, 0] = 1
        nearinfo[:, 1] = latnear
        nearinfo[:, 2] = lonnear
        nearinfo[:, 3] = demnear

        tarinfo = np.zeros(4)
        tarinfo[0] = 1
        tarinfo[1] = lattar[rr]
        tarinfo[2] = lontar[cc]
        tarinfo[3] = demtar[rr, cc]

        tx_red = np.transpose(nearinfo)
        twx_red = np.matmul(tx_red, weightnear)

        for tt in range(ntimes):
            datanear = dataori[rloc[0]:rloc[1] + 1, cloc[0]:cloc[1] + 1, tt]
            datanear = np.reshape(datanear, nnum)

            # upper and lower boundary for the downscaled data
            # this is a conservative limitation
            lowbound = np.min(datanear)
            upbound = np.max(datanear)

            b = reg.least_squares(nearinfo, datanear, twx_red)
            datatemp = np.dot(tarinfo, b)
            if np.all(b == 0) or datatemp > upbound or datatemp < lowbound:
                # use nearest neighbor interpolation
                weightnear = weight[rr, cc, 0:nnum]
                mloc = np.argmax(weightnear)
                datatar[gg, tt] = datanear[mloc]
            else:
                datatar[gg, tt] = datatemp
    return datatar

def readstndata(inpath,stnID,ndays):
    nstn = len(stnID)
    prcp_stn = np.nan * np.zeros([nstn,ndays])
    tmin_stn = np.nan * np.zeros([nstn, ndays])
    tmax_stn = np.nan * np.zeros([nstn, ndays])


    for i in range(nstn):
        if np.mod(i,500) == 0:
            print('station',i,nstn)

        file = inpath + '/' + stnID[i] + '.nc'
        fid = nc.Dataset(file)
        varlist = fid.variables.keys()
        if 'prcp' in varlist:
            prcp_stn[i,:] = fid['prcp'][:].data
        if 'tmin' in varlist:
            tmin_stn[i,:] = fid['tmin'][:].data
        if 'tmax' in varlist:
            tmax_stn[i,:] = fid['tmax'][:].data
        fid.close()

    tmean_stn = (tmin_stn+tmax_stn)/2
    trange_stn = np.abs(tmax_stn - tmin_stn)

    return prcp_stn, tmean_stn, trange_stn


# input parameters
# a = int(sys.argv[1])
# b = int(sys.argv[2])
# year = [a, b]
# year = [1979, 1979]
print('year',year)

# basic information
# mac
filedem = './DEM/NA_DEM_010deg_trim.mat'
# plato
# filedem = '/datastore/GLOBALWATER/CommonData/EMDNA/DEM/NA_DEM_010deg_trim.mat'
vars = ['prcp', 'tmin', 'tmax']
lontar = np.arange(-180 + 0.05, -50, 0.1)
lattar = np.arange(85 - 0.05, 5, -0.1)
hwsize = 2

datatemp = io.loadmat(filedem)
demtar = datatemp['DEM']  # this is consistent with lontar lattar
mask = demtar.copy()
mask[~np.isnan(mask)] = 1

# station information
gmet_stnfile = '/Users/localuser/GMET/pyGMET_NA/stnlist_whole.txt'
gmet_stnpath = '/Users/localuser/GMET/StnInput_daily'
gmet_stndata = '/Users/localuser/GMET/pyGMET_NA/stndata_whole.npz'
stn_ID = np.genfromtxt(gmet_stnfile, dtype='str', skip_header=1, comments='#', delimiter=',', usecols=(0), unpack=False)
stn_lle = np.loadtxt(gmet_stnfile, dtype=float, skiprows=1, comments='#', delimiter=',', usecols=(1,2,3), unpack=False)
stn_row = ((85 - stn_lle[:,0]) / 0.1).astype(int)
stn_col = ((stn_lle[:,1] + 180) / 0.1 ).astype(int)
if not os.path.isfile(gmet_stndata):
    prcp_stn, tmean_stn, trange_stn = readstndata(gmet_stnpath, stn_ID, 14610)
    np.savez_compressed(gmet_stndata, prcp_stn=prcp_stn, tmean_stn=tmean_stn, trange_stn=trange_stn,
                        stn_ID=stn_ID, stn_lle=stn_lle, stn_row=stn_row, stn_col=stn_col)

year [1979, 1979]


In [179]:
filedem_era = './DEM/ERA5_DEM2.mat'
inpath = '/Users/localuser/Research/Test'
outpath = '/Users/localuser/Research'
filenear = outpath + '/weight_dem.npz'

for y in range(year[0], year[1] + 1):
    for v in range(1):
        print('year--var:', y, vars[v])
        infile = inpath + '/ERA5_' + vars[v] + '_' + str(y) + '.mat'
        outfile_grid = outpath + '/ERA5_' + vars[v] + '_' + str(y) + '.npz'
        outfile_stn = outpath + '/ERA5_stn_' + vars[v] + '_' + str(y) + '.npz'
        if os.path.isfile(outfile_grid) and os.path.isfile(outfile_stn):
            continue

        # load original daily reanalysis data
        datatemp = {}
        f = h5py.File(infile,'r')
        for k, v in f.items():
            datatemp[k] = np.array(v)
        latori = datatemp['latitude'][0]
        lonori = datatemp['longitude'][0]
        dataori = datatemp['data']
        dataori = np.transpose(dataori, [2, 1, 0])
        del datatemp
        f.close()

        # read location information
        if not os.path.isfile(filenear):
            rowse, colse, weight = neargrid(lattar, lontar, latori, lonori, hwsize)
            # extract ori dem
            demori = demread(filedem_era, latori, lonori)
            io.savemat(filenear, {'rowse': rowse, 'colse': colse, 'weight': weight, 'demori': demori})
        else:
            datatemp = io.loadmat(filenear)
            rowse = datatemp['rowse']
            colse = datatemp['colse']
            weight = datatemp['weight']
            demori = datatemp['demori']
            del datatemp
        
        if not os.path.isfile(outfile_stn):
            # downscale the reanalysis to 0.1 degree
            datatar = readownscale_tostn(dataori, latori, lonori, demori, lattar, lontar, demtar, rowse, colse, weight, stn_row, stn_col)
            datatar = np.float32(datatar)
            np.savez_compressed(outfile_stn, data=datatar, latitude=lattar, longitude=lontar,
                                stn_ID=stn_ID, stn_lle=stn_lle, stn_row=stn_row, stn_col=stn_col)

year--var: 1979 prcp
station 0 27275
station 500 27275
station 1000 27275
station 1500 27275
station 2000 27275
station 2500 27275
station 3000 27275
station 3500 27275
station 4000 27275
station 4500 27275
station 5000 27275
station 5500 27275
station 6000 27275
station 6500 27275
station 7000 27275
station 7500 27275
station 8000 27275
station 8500 27275
station 9000 27275
station 9500 27275
station 10000 27275
station 10500 27275
station 11000 27275
station 11500 27275
station 12000 27275
station 12500 27275
station 13000 27275
station 13500 27275
station 14000 27275
station 14500 27275
station 15000 27275
station 15500 27275
station 16000 27275
station 16500 27275
station 17000 27275
station 17500 27275
station 18000 27275
station 18500 27275
station 19000 27275
station 19500 27275
station 20000 27275
station 20500 27275
station 21000 27275
station 21500 27275
station 22000 27275
station 22500 27275
station 23000 27275
station 23500 27275
station 24000 27275
station 24500 27275
sta

In [185]:
datatemp = np.load(gmet_stndata)
prcp_stn0=datatemp['prcp_stn'][:,0]

In [187]:
len(prcp_stn0)

27275